# Dev Environment Setup [Ubuntu/Mint]

**Correct 2016-09 - to be verified and revised 2017-03**

This assumes a brand new install of Ubuntu 16.04 or Mint 18. If this is not the case and you already have some of the below set up, in particular MariaDB/MySQL, skip the installation steps for those dependencies.

---

First, download and install [RVM](https://rvm.io/) if it isn't already present on your development machine, then Ruby 2.1.1.

Do not install RVM as Root; when it needs `sudo` access it will ask for it.

```
gpg --keyserver hkp://keys.gnupg.net --recv-keys 409B6B1796C275462A1703113804BB82D39DC0E3

\curl -sSL https://get.rvm.io | bash -s stable

rvm install 2.1.1
```

`rvm pkg` is deprecated, instead set rvm to take care of missing dependencies where necessary:

```
rvm autolibs packages
```

Install the dependencies for MySQL, RMagick and Nokogiri (Nokogiri 1.5.2 is required, but predates Nokogiri vendoring its own dev dependencies):

```
sudo apt-get install mariadb-server mariadb-client graphicsmagick-libmagick-dev-compat imagemagick libxslt1-dev libxml2-dev
```

Note that we are NOT using libmagick9-dev - this has been deprecated due to serious security flaws. If you are on Ubuntu, Aptitude should give you a warning about this.

We need to override some of that, and install the core and 'optional' packages:

```
sudo apt-get install libmagickcore-dev
sudo apt-get install libmagickwand-dev
```

Then link the necessary binary to the PATH:

```
sudo ln -s /usr/lib/x86_64-linux-gnu/ImageMagick-6.8.9/bin-Q16/Magick-config /usr/bin/Magick-config
```

We also need to finish setting up MySQL; enter the following and accept all the defaults. Default root password is blank:

```
sudo /usr/bin/mysql_secure_installation

sudo mysql -u root
sudo mysql -u root -p

CREATE USER 'marketdojo'@'localhost' IDENTIFIED BY '[password here]';
GRANT ALL PRIVILEGES ON *.* TO 'marketdojo'@'localhost' WITH GRANT OPTION;
\q
```

Clone the repository - you should have been given access via GitHub invite already. If not, contact Nic Martin for access:

```
git clone git@github.com:niciliketo/auction.git

cd auction
```

RVM should detect the `.ruby_gemset` file, and create an empty `lot_structure` gemset. Switch to that if necessary:

```
rvm use 2.1.1@lot_structure
```

Install Bundler as the first gem, followed by the remaining gems:

```
gem install bundler

bundle install
```

`config/database.yml` is in the .gitignore, so you will need to create one. The following template works:

```

default: &default
  adapter: mysql2
  encoding: utf8
  username: marketdojo
  password: [password created for MySQL above]
  pool: 5
  timeout: 5000

development:
  <<: *default
  database: auction_development

test:
  <<: *default
  database: marketdojo_test

production:
  <<: *default
  database: marketdojo_production

```

You should be able to create the databases:

```
rake db:create
rake db:schema:load
```

Running `rails s` should now run the site at http://localhost:3000

There is no data in the database initially; testing data will need to be acquired from another team member. This will take the form of a gzipped dump of the current database. Unzip it, change the file extension from .bak to .sql, and then run `mysql -u root -p auction_development < [filename.sql]`

For running tests (Test::Unit and RSpec) we use Parallel Tests, with fixtures for some of the necessary data. This needs a little further setting up:

```
rake parallel:create
rake parallel:prepare (This **must** be run after any migrations; it ensures all your test databases have the same schema as your development DB!)
```

Tests are then run with either `rake parallel:test` for Test::Unit, or `rake parallel:spec` for RSpec. You should run `rake test` before trying anything else, as this loads the necessary fixtures.

---
# Dev Environment Setup [OSX 10.11]

**Correct 2016-09 - to be verified and revised 2017-03**

This assumes a brand new install of OSX 10.11 'El Capitan'. If this is not the case and you already have some of the below set up, in particular MariaDB/MySQL, skip the installation steps for those dependencies.

---

A new install of OSX 10.11 will have a system Ruby, but it is outdated.

For development work, XCode will be necessary:

```
xcode-select --install
```

The Homebrew package manager will be required for later steps:

```
ruby -e "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/master/install)"

brew install caskroom/cask/brew-cask
```

For managing Ruby and Gemsets we need RVM, but prior to that we require GNUPG; this is not included in OSX by default, unlike in Ubuntu.

```
brew install gnupg gnupg2

gpg --keyserver hkp://keys.gnupg.net --recv-keys 409B6B1796C275462A1703113804BB82D39DC0E3

\curl -sSL https://get.rvm.io | bash -s stable

rvm install 2.1.1
```

`rvm pkg` is deprecated, instead set rvm to take care of missing dependencies where necessary:

```
rvm autolibs packages
```

We use MySQL (in fact MariaDB, but there is deliberately no difference in any of the commands) for the database backend:

```
brew install mariadb
unset TMPDIR
mysql_install_db
```

Ensure that the process `mysqld` is running, then finish the setup:

```
brew services start mariadb
mysql_secure_installation

sudo mysql -u root -p

CREATE USER 'marketdojo'@'localhost' IDENTIFIED BY '[password here]';
GRANT ALL PRIVILEGES ON *.* TO 'marketdojo'@'localhost' WITH GRANT OPTION;
\q
```

Nokogiri and RMagick both require native extensions:

```
brew install libxml2 libxslt openssl
brew install homebrew/dupes/libiconv

brew install ImageMagick
```


Git is required for the workflow; `brew install git` takes care of that and its dependencies.

Assuming you already have a GitHub account with an SSH key:

```
git clone git@github.com:niciliketo/auction.git
cd auction
```

RVM should automatically create the `lot_structure` gemset. To have most of the dev commands work, you will need bundler; use that to install the remainder of the required gems.

```
gem install bundler

bundle config build.puma --with-opt-include=/usr/local/opt/openssl/include
```
This is necessary because our version of Puma requires a version of openssl which Apple has chosen to override. 

```
bundle install
```

All being well, this should install all the Gem dependencies.

`config/database.yml` is in the .gitignore, so you will need to create one. The following template works:

```

default: &default
  adapter: mysql2
  encoding: utf8
  username: marketdojo
  password: [password created for MySQL above]
  pool: 5
  timeout: 5000

development:
  <<: *default
  database: auction_development

test:
  <<: *default
  database: marketdojo_test

production:
  <<: *default
  database: marketdojo_production

```

You should be able to create the databases:

```
rake db:create
rake db:schema:load
```

Running `rails s` should now run the site at http://localhost:3000

There is no data in the database initially; testing data will need to be acquired from another team member. This will take the form of a gzipped dump of the current database. Unzip it, change the file extension from .bak to .sql, and then run `mysql -u root -p auction_development < [filename.sql]`

For running tests (Test::Unit and RSpec) we use Parallel Tests, with fixtures for some of the necessary data. This needs a little further setting up:

```
rake parallel:create
rake parallel:prepare (This **must** be run after any migrations; it ensures all your test databases have the same schema as your development DB!)
```

Tests are then run with either `rake parallel:test` for Test::Unit, or `rake parallel:spec` for RSpec. You should run `rake test` before trying anything else, as this loads the necessary fixtures.